In [49]:
from tensorflow import keras
from tf_keras_vis.activation_maximization import ActivationMaximization
from tensorflow import random

In [50]:
from tf_keras_vis.activation_maximization.regularizers import Norm, TotalVariation2D
from tf_keras_vis.activation_maximization.input_modifiers import Jitter, Rotate2D, Scale
from tensorflow.keras.optimizers import RMSprop, Adam

In [51]:
model =  keras.models.load_model("model-files")

In [52]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 244, 244, 64)      9472      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 122, 122, 64)     0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 116, 116, 32)      100384    
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 58, 58, 32)       0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 52, 52, 32)        50208     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 26, 26, 32)      

In [53]:
from tf_keras_vis.utils.model_modifiers import ExtractIntermediateLayer, ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore
from tf_keras_vis.activation_maximization import ActivationMaximization
from tf_keras_vis.activation_maximization.callbacks import Progress, GifGenerator2D

max_filters = 20

layer_names = ["conv2d_13", "conv2d_14"]
activations = {}
for layer_name in layer_names:
    n_filters = min(model.get_layer(layer_name).filters, max_filters)

    extract_intermediate_layer = ExtractIntermediateLayer(index_or_name=layer_name)
    replace2linear = ReplaceToLinear()
    activation_maximization = ActivationMaximization(model,model_modifier=[extract_intermediate_layer, replace2linear],clone=True)
    
    seed_input_shape  = (n_filters, model.input_shape[1], model.input_shape[2],model.input_shape[3])
    seed_input = random.uniform(seed_input_shape, 0, 255)
    scores = CategoricalScore(list(range(n_filters)))

    regularizers=[TotalVariation2D(weight=1), Norm(weight=0.1, p=1)]
    input_modifiers = [Jitter(jitter=1), Rotate2D(degree=2)],
    optimizer = RMSprop(learning_rate = 0.1)
    activations[layer_name] = activation_maximization(scores,
                                                    steps = 200 ,
                                                    regularizers=regularizers,
                                                    optimizer = optimizer,
                                                    input_modifiers = input_modifiers,
                                                    seed_input = seed_input, 
                                                    callbacks=[Progress(), GifGenerator2D("gif/optimizer.gif")])


200/200 [==============================] - 680s 3s/step - Score: 677.4527 - TotalVariation2D: 4.8023 - Norm: 12.7532


In [55]:
from numpy import savez_compressed
for layer_name in layer_names:
    savez_compressed("activation-files/activations_" + layer_name +'.npz', activations[layer_name])